# fine-Tuning (LoRA)

In [ ]:
# !pip install transformers datasets peft trl accelerate bitsandbytes

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

token = "your_token"

In [ ]:
# ✅ 1. 데이터셋 로드 (train, validation)
dataset = load_dataset("json", data_files={
    "train": "preprocessing/new/train/gap-dev_npe_sft.jsonl",
    "validation": "./preprocessing/gap/gap-validation_sft.jsonl"
})

# ✅ 2. Tokenizer 및 모델 로드 (패딩 토큰, 8bit 양자화)
model_name = "google/gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
tokenizer.pad_token = tokenizer.eos_token  # Gemma는 pad_token이 없음 → eos로 대체

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",  # GPU 자동 할당
    token=token
)

# ✅ 3. LoRA 설정
lora_config = LoraConfig(
    r=16,               # 랭크 (더 높이면 성능↑, 낮으면 속도↑)
    lora_alpha=32,
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

# ✅ 4. 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# ✅ 5. 학습 파라미터 (메모리, 속도 고려 최적화)
training_args = TrainingArguments(
    output_dir="./finetuned/gemma-lora-2-9b_e10-gap-dev_npe",  # 저장 경로
    per_device_train_batch_size=4,       # GPU 메모리 고려
    gradient_accumulation_steps=2,       # 가상 배치 = 16
    learning_rate=2e-5,
    num_train_epochs=10,
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    save_total_limit=2,                  # 최대 2개 checkpoint만 유지
    load_best_model_at_end=True,         # 가장 성능 좋은 모델 복원
    metric_for_best_model="eval_loss",   # 평가지표
    greater_is_better=False,             # loss 작을수록 좋음
    bf16=True,                          # RTX 6000 Ada 지원
    optim="paged_adamw_8bit",             # 8bit 최적화 옵티마이저
    warmup_steps=50,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    report_to="none"                     # wandb 사용 안함
)

In [ ]:
# ✅ 6. SFTTrainer 세팅
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer
)

# ✅ 7. 학습 시작
trainer.train()

# ✅ 8. LoRA 어댑터만 저장 (base model 제외)
trainer.model.save_pretrained("./finetuned/gemma-lora-2-9b_e10-gap-dev_npe")

# inference

In [ ]:
import os
import json
import time
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import re

In [ ]:
token = "your_token"

# ✅ base model (Gemma) 로드
base_model_name = "google/gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, token=token)
tokenizer.pad_token = tokenizer.eos_token  # 패딩 없을 때 필수
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    token=token
)

adpater적용시

In [ ]:
# adapter_path = "./finetuned/gemma/gemma-lora-2-9b_e10-gap-dev_npe"  # 학습한 adapter 경로
# model = PeftModel.from_pretrained(model, adapter_path, token=token)  # LoRA 적용

In [ ]:
# ✅ 디버깅 포함된 추론 함수
def query_Gemma_debug(prompt, max_new_tokens=100):

    try:
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

        outputs = model.generate(
            **input_ids,
            max_new_tokens=max_new_tokens,
            temperature=0.0,  # 확정적
            top_p=1.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        # ✅ 프롬프트 제거
        if prompt in response:
            response = response.replace(prompt, "").strip()

        # ✅ 'Answer:' 이후만 추출
        if 'Answer:' in response:
            response = response.split('Answer:')[-1].strip()

        if not response:
            return "API FAILED"
        return response

    except Exception as e:
        print(f"❌ [에러] 모델 호출 실패: {e}")
        return "API FAILED"



def clean_answer_debug(response):
    pattern = re.compile(r'^(A|B|Neither)\b', re.IGNORECASE)  # 문장 맨 앞에 등장하는 A/B/Neither만
    match = pattern.search(response)
    if match:
        return match.group(1).upper()
    else:
        print("[정답 패턴 없음]")
        return "INVALID"


In [ ]:
name_list = ["wsc"]  # 사용할 데이터셋 이름

# ✅ 메인 실행
for name in name_list:
    json_file_path = os.path.join(os.getcwd(), "preprocessing", "test","wsc", f"{name}.json")
    csv_file_path = os.path.join(os.getcwd(), "output", "zero", "Gemma-zero", f"{name}.csv")  # 저장 파일명 수정

    # 데이터 로드
    with open(json_file_path, "r", encoding="utf-8") as json_file:
        test_data = json.load(json_file)

    # 기존 처리된 데이터 확인
    if os.path.exists(csv_file_path):
        df_existing = pd.read_csv(csv_file_path, encoding="utf-8")
        processed_ids = set(df_existing["text_id"].tolist())
        print(f"🔄 기존 데이터 {len(processed_ids)}개 로드 완료. 이어서 실행합니다.")
    else:
        df_existing = pd.DataFrame()
        processed_ids = set()

    file_exists = os.path.exists(csv_file_path)  # 헤더 결정

    # ✅ 데이터 순회
    for data in test_data:
        if data["text_id"] in processed_ids:
            continue  # 이미 처리된 경우 스킵

        # 프롬프트
#         prompt = f'''Question: In the sentence "{data["text"]}", what does "{data["target"]}" refer to?
# Options:
# (A) {data["options"]["A"]}
# (B) {data["options"]["B"]}

# Answer only with "A" if (A) is correct, "B" if (B) is correct, or "Neither" if none of them are correct. Do not provide explanations.
# Answer:'''
        
        prompt = f'''Question: In the sentence "{data["text"]}", what should replace "{data["target"]}"?
Options:
(A) {data["options"]["A"]}
(B) {data["options"]["B"]}
Answer only with "A" if (A) is correct, "B" if (B) is correct, or "Neither" if none of them are correct. Do not provide explanations.
Answer:'''

        # 모델 호출 및 디버깅
        response = query_Gemma_debug(prompt)
        answer = clean_answer_debug(response)

        # 정답 비교
        correct = (answer == data["answer"].strip().upper())

        # ✅ 결과 저장
        result = {
            "text_id": data["text_id"],
            "text": data["text"],
            "target": data["target"],
            "expected_answer": data["answer"].strip().upper(),
            "Gemma_LoRA_answer": answer,
            "correct": correct
        }
        # CSV 저장
        df_temp = pd.DataFrame([result])
        df_temp.to_csv(csv_file_path, mode="a", index=False, header=not file_exists, encoding="utf-8")
        file_exists = True

    print(f"✅ [{name}] 모든 데이터 처리 완료: {csv_file_path}")
